This is the for connecting to a pressure sensor 
at the bottom of a lake and logging it to its own 
memory.

For now it will do it to MQTT

In [124]:
%serialconnect

Connecting to --port=/dev/ttyUSB7 --baud=115200 
Ready.


In [125]:
%sendtofile main.py

import machine, time, network, os

sleepperiod = const(1)
mqtt_local = "10.0.29.167"


#
# Get the RTC working
#
i2c = machine.I2C(scl=machine.Pin(4), sda=machine.Pin(5)) 
epoch1970 = const(946684800000)   # esp8266 has 2000 epoch
def rhex(h):  return (h>>4)*10 + (h&0x0f)
def rtctojsepoch():
    r = i2c.readfrom_mem(0x68, 0x00, 7)
    year, month, day = rhex(r[6])+2000, rhex(r[5]), rhex(r[4])
    hour, minute, second = rhex(r[2]), rhex(r[1]), rhex(r[0])
    microsecond = 0
    micropythonepoch = time.mktime((year, month, day, hour, minute, second, -1, -1))
    return micropythonepoch*1000 + microsecond + epoch1970   # esp8266 has 2000 

def jsepochtoisodate(jsepoch):
    micropythonepoch = (jsepoch - epoch1970)//1000
    microsecond = jsepoch % 1000
    year, month, day, hour, minute, second, d1, d2 = time.localtime(micropythonepoch)
    return "{0:04d}-{1:02d}-{2:02d}T{3:02d}:{4:02d}:{5:02d}.{6:03d}".format(year, month, day, hour, minute, second, microsecond)

#
# make name of datafile
#
for i in range(100):
    fname = "data%03d.txt" % i
    try:     os.stat(fname)
    except OSError:  break
print(fname)

#
# connect MQTT
#
sta_if = network.WLAN(network.STA_IF)
sta_if.active(True)
sta_if.connect("DoESLiverpool", "decafbad00")
while not sta_if.isconnected():
    pass

print("connected to wifi")
from umqtt.robust import MQTTClient
client = MQTTClient("lakelogger", mqtt_local)
client.connect()
print("connected to mqtt")


#
# Connect to ADC
#
adc = machine.ADC(0)


#
# Main working loop
#
n = 0
fout = None
while True:
    n += 1
    aval = adc.read()
    if fout is None:
        fout = open(fname, "aw")
    client.publish(b"lakelogger/level", str(aval))
    try:
        ts = jsepochtoisodate(rtctojsepoch())
    except OSError as e:
        ts = "unknown"
    fout.write("%s %d\n" % (ts, aval))
    if (n%10) == 0:
        fout.flush()
        fout.close()
        fout = None

    time.sleep(sleepperiod)
    

Sent 80 lines (1938 bytes) to main.py.


serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB7 --baud=115200 
Ready.


In [127]:
# Check what files we have
%serialconnect
import os
for f in os.ilistdir():
    if f[0][0] == "d":
        print(f[0], f[3])


Connecting to --port=/dev/ttyUSB7 --baud=115200 
Ready.
data000.txt 0


In [126]:
%disconnect

attempt to exit paste mode
[\r\x03\x02] b'\r\nMicroPython v1.9.4-8-ga9a3caad0 on 2018-05-11; ESP module with ESP8266\r\nType "help()" for more information.\r\n>>> '
Closing serial Serial<id=0x7f72a1c13390, open=True>(port='/dev/ttyUSB7', baudrate=115200, bytesize=8, parity='N', stopbits=1, timeout=0.5, xonxoff=False, rtscts=False, dsrdtr=False)


In [103]:
%fetchfile --binary data000.txt

Fetched 22680=22680 bytes from data000.txt.
Saving file to 'data000.txt'

In [238]:
%serialconnect

Connecting to --port=/dev/ttyUSB6 --baud=115200 
Ready.


In [211]:
import machine
print(machine.freq())

80000000


In [239]:
%sendtofile main.py

#
# Demo of deep sleep cycle (pin16 to RST on ESP8266)
#
import machine
st = str(machine.RTC().datetime())
f = open("log.txt", "a")
f.write(st)
print(st)
if machine.reset_cause() == machine.DEEPSLEEP_RESET:
    f.write('  woke from a deep sleep\n')
else:
    f.write('  power on or hard reset\n')
f.close()

import time
p = machine.Pin(4, machine.Pin.OUT)
for i in range(20):
    p.value(i%2)
    time.sleep(0.2)
p.init(machine.Pin.IN)

# configure RTC.ALARM0 to be able to wake the device
machine.deepsleep(sleep_ms=5000)


Sent 23 lines (523 bytes) to main.py.


In [233]:
%serialconnect

Connecting to --port=/dev/ttyUSB6 --baud=115200 
Ready.


In [229]:
#%sendtofile main.py

import time, machine
#p = machine.Pin(2, machine.Pin.OUT)
p = machine.Pin(4, machine.Pin.OUT)
for i in range(221):
    p.value(i%2)
    time.sleep(0.6)




*** Sending Ctrl-C



Traceback (most recent call last):
  File "<stdin>", line 8, in <module>
KeyboardInterrupt: 


In [230]:
p.init(machine.Pin.IN)

In [177]:
for l in open("log.txt"):
    print(l, end="")

Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
OSError: [Errno 2] ENOENT


In [161]:
%disconnect

attempt to exit paste mode
[\r\x03\x02] b'\r\nMicroPython v1.9.4-8-ga9a3caad0 on 2018-05-11; ESP module with ESP8266\r\nType "help()" for more information.\r\n>>> '
Closing serial Serial<id=0x7f72a1c133c8, open=True>(port='/dev/ttyUSB7', baudrate=115200, bytesize=8, parity='N', stopbits=1, timeout=0.5, xonxoff=False, rtscts=False, dsrdtr=False)


In [ ]:
#
# Call this to set the time on the external RTC
#
import ure
def dhex(v):  return chr(((v//10)<<4) + (v%10))
def jsepochtortc(jsepoch):
    micropythonepoch = (jsepoch - epoch1970)//1000
    microsecond = jsepoch % 1000
    year, month, day, hour, minute, second, d1, d2 = time.localtime(micropythonepoch)
    i2c.writeto(0x68, chr(0) + dhex(second) + dhex(minute) + dhex(hour))
    i2c.writeto(0x68, chr(4) + dhex(day) + dhex(month) + dhex(year-2000))
def isodatetojsepoch(isodate):
    mtime = ure.match("(\d\d\d\d)-(\d\d)-(\d\d)[T ](\d\d)[:c](\d\d)[:c](\d\d)[\.d](\d\d\d)", isodate)
    year, month, day, hour, minute, second, microsecond = int(mtime.group(1)), int(mtime.group(2)), int(mtime.group(3)), int(mtime.group(4)), int(mtime.group(5)), int(mtime.group(6)), int(mtime.group(7)) 
    micropythonepoch = time.mktime((year, month, day, hour, minute, second, -1, -1))
    return micropythonepoch*1000 + microsecond + epoch1970
#jsepochtortc(isodatetojsepoch("2018-11-16T13:00:00.000"))
